<a href="https://colab.research.google.com/github/ayyucedemirbas/kvasir_seg_self-supervised/blob/main/kvasir_seg_ssl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
import tensorflow.keras as keras
import os
import zipfile
import numpy as np
import random
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0

In [8]:
!wget https://datasets.simula.no/downloads/kvasir-seg.zip
!unzip -qq kvasir-seg.zip

--2024-12-10 23:14:01--  https://datasets.simula.no/downloads/kvasir-seg.zip
Resolving datasets.simula.no (datasets.simula.no)... 128.39.36.14
Connecting to datasets.simula.no (datasets.simula.no)|128.39.36.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46227172 (44M) [application/zip]
Saving to: ‘kvasir-seg.zip’

kvasir-seg.zip      100%[===================>]  44.08M  14.1MB/s    in 3.1s    

2024-12-10 23:14:04 (14.1 MB/s) - ‘kvasir-seg.zip’ saved [46227172/46227172]



In [11]:
def preprocess_image(image_path):
    """Preprocess the input image."""
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image

def data_augment(image):
    """Apply augmentations for contrastive learning."""
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_crop(image, size=(200, 200, 3))
    image = tf.image.resize(image, (224, 224))
    return image

def contrastive_data_loader(image_paths, batch_size):
    """Load and prepare contrastive data."""
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(lambda x: (data_augment(x), data_augment(x)),
                          num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset


In [12]:
DATASET_PATH = "/content/Kvasir-SEG"

image_paths = tf.io.gfile.glob(os.path.join(DATASET_PATH, "images", "*.*"))
random.shuffle(image_paths)

BATCH_SIZE = 32
train_dataset = contrastive_data_loader(image_paths, batch_size=BATCH_SIZE)

# Define the SimCLR model backbone
def create_simclr_model():
    """Create a SimCLR-style model."""
    base_model = EfficientNetB0(include_top=False, weights=None, input_shape=(224, 224, 3))
    base_model.trainable = True

    inputs = tf.keras.Input(shape=(224, 224, 3))
    features = base_model(inputs, training=True)
    pooled_features = GlobalAveragePooling2D()(features)
    projection_head = Dense(128, activation="relu")(pooled_features)
    projection_head = Dense(128)(projection_head)

    model = tf.keras.Model(inputs, projection_head)
    return model

simclr_model = create_simclr_model()



In [18]:
cosine_similarity = keras.losses.CosineSimilarity(axis=-1, reduction=tf.keras.losses.Reduction.NONE)


def nt_xent_loss(features):
    """Normalized Temperature-scaled Cross Entropy Loss."""
    batch_size = tf.shape(features)[0] // 2

    labels = tf.eye(batch_size * 2)
    masks = tf.eye(batch_size * 2)

    similarities = tf.linalg.matmul(features, features, transpose_b=True) / 0.5
    similarities -= masks * 1e9

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels, similarities))
    return loss

optimizer = keras.optimizers.Adam(learning_rate=1e-3)



In [ ]:
EPOCHS = 100
for epoch in range(EPOCHS):
    epoch_loss = 0.0
    for step, (img_1, img_2) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            features_1 = simclr_model(img_1, training=True)
            features_2 = simclr_model(img_2, training=True)

            features = tf.concat([features_1, features_2], axis=0)
            loss = nt_xent_loss(features)

        gradients = tape.gradient(loss, simclr_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, simclr_model.trainable_variables))

        epoch_loss += loss

    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {epoch_loss / (step + 1):.4f}")


Epoch 1/100, Loss: 999999616.0000
Epoch 2/100, Loss: 999999104.0000


In [ ]:
simclr_model.save("simclr_kvasir_pretrained_model")